In [1]:
import numpy as np
import h5py
import os,sys
from matplotlib import pyplot as plt

In [12]:
def readh5(filename, datasetname='main', rr=[1,1,1]):
    fid=h5py.File(filename,'r')
    if isinstance(datasetname, (list,)):
        out = [None] *len(datasetname)
        for i,dd in enumerate(datasetname):
            sz = len(fid[dd].shape)
            if sz==2:
                out[i] = np.array(fid[dd][::rr[0],::rr[1]])
            elif sz==3:
                out[i] = np.array(fid[dd][::rr[0],::rr[1],::rr[2]])
    else:
        sz = len(fid[datasetname].shape)
        if sz==2:
            out = np.array(fid[datasetname][::rr[0],::rr[1]])
        elif sz==3:
            out = np.array(fid[datasetname][::rr[0],::rr[1],::rr[2]])
        elif sz==4:
            out = np.array(fid[datasetname][::,::rr[0],::rr[1],::rr[2]])
    return out

def writeh5(filename, dtarray, datasetname='main'): 
    fid=h5py.File(filename,'w')                                                                      
    if isinstance(datasetname, (list,)):                                                             
        for i,dd in enumerate(datasetname):                                                          
            ds = fid.create_dataset(dd, dtarray[i].shape, compression="gzip", dtype=dtarray[i].dtype)
            ds[:] = dtarray[i]                                                                       
    else:                                                                                            
        ds = fid.create_dataset(datasetname, dtarray.shape, compression="gzip", dtype=dtarray.dtype) 
        ds[:] = dtarray                                                                              
    fid.close()

def processLabels(label_volume, bv_labels):
    """
    The goal is to make backgound:0, bloodvessel:1, cellbody:2
    """
    Z, X, Y = label_volume.shape
    
    for z in range(Z):
        for x in range(X):
            for y in range(Y):
                if label_volume[z,x,y] in bv_labels:
                    label_volume[z,x,y] = 1
                elif label_volume[z,x,y] > 0:
                    label_volume[z,x,y] = 2
    return label_volume

In [3]:
v1_bv_label = [494, 489, 514, 486, 476, 521, 522, 495, 510]
v2_bv_label = [1,2,3,4,5,8,10,15,16] # Manually examined from neuro glancer.

In [4]:
v1_label = readh5("V1/V1_anno_dense_bv_cell.h5")

In [5]:
v1_label_corrected = processLabels(v1_label, v1_bv_label)

In [6]:
np.unique(v1_label_corrected)

array([0, 1, 2])

In [13]:
writeh5("V1_anno_dense_bv_cell_corrected.h5", v1_label_corrected)

In [7]:
v2_label = readh5("V2/V2_anno_dense_bv_cell.h5")

In [8]:
v2_label_corrected = processLabels(v2_label, v2_bv_label)

In [9]:
np.unique(v2_label_corrected)

array([0, 1, 2])

In [14]:
writeh5("V2_anno_dense_bv_cell_corrected.h5", v2_label_corrected)